In [1]:
from modelArchitecture.ResnetRnnDense import ResnetRnnDense
from modelArchitecture.VirtualBatchNormalization import VirtualBatchNormalization
from modelArchitecture.Attention import Attention
from keras.models import model_from_yaml
from utils import split_data, translate, save_model, standardize, result
from HydraMethod import HydraMethod
import keras
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.losses import mean_squared_error
import math
import numpy as np
from keras.backend import eval

PATH = "./data/"
splitter = split_data(PATH + "raw.csv", PATH, test_size=0.1, train_size=0.9)
x_train, y_train, x_test, y_test = splitter.split()
translater = translate()

x_train = translater.translate(x_train)
x_test = translater.translate(x_test)

y_train_std, y_test_std = standardize(y_train, y_test)

/home/markus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
hydra = HydraMethod(ResnetRnnDense(),4)

In [3]:
x_train, x_test = hydra.get_all_representation(x_train), hydra.get_all_representation(x_test)

In [4]:
class LR():
    def __init__(self, lr):
        self.lr = lr

    def step_decay(self, epoch):
        initial_lrate = self.lr
        drop = 0.4
        epochs_drop = 10.0
        lrate = initial_lrate * math.pow(drop,
                                         math.floor((1+epoch)/epochs_drop))
        print(lrate)
        return lrate

In [5]:
def freeze_it(model, block_nr):
    counter = 0
    for layer in model.layers:
        if "add" in layer.name:
            counter += 1
        if counter >= 4 * block_nr:
            layer.trainable = True
        else:
            layer.trainable = False
    print(counter / 4)
    return model

In [6]:
def T95(model, x, y):
    predict_train = model.predict(x)
    res_train = result(y, predict_train)
    print(np.round(res_train.t95()[0], 4))

In [7]:
EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=0, patience=6,
                                              verbose=1,
                                              restore_best_weights=True)

In [8]:
def training(model, X_train, y_train_std, X_test, y_test_std, lr, bz, k=None):
    if k:
        print("not none")
        model = freeze_it(model, k)

    lr = LR(lr)
    lrate = LearningRateScheduler(lr.step_decay)

    model.compile(loss=mean_squared_error, optimizer=Adam(7e-5))
    history = model.fit(X_train, y_train_std, validation_data=(X_test, y_test_std),
              epochs=2, batch_size=bz, callbacks=[lrate, EarlyStopping])



    T95(model,X_train,y_train_std)
    T95(model,X_test,y_test_std)
    return model


In [9]:
from keras.models import model_from_json

yaml_file = open('models/LRF_model.json', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()

model = model_from_json(loaded_model_yaml,
                        custom_objects={'VirtualBatchNormalization':VirtualBatchNormalization, 'Attention':Attention})

model.load_weights('models/LRF_model.h5')

32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32


In [10]:
lr = 5e-4
model = training(model,x_train,y_train_std,x_test,y_test_std,lr,256)

/home/markus/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 968303 samples, validate on 110666 samples
Epoch 1/2
0.0005
968303/968303 [==============================] - 292s 302us/step - loss: 0.0450 - val_loss: 0.0263
Epoch 2/2
0.0005
968303/968303 [==============================] - 279s 288us/step - loss: 0.0280 - val_loss: 0.0380
12.6842
12.6838


In [11]:
#lr = eval(model.optimizer.lr) * 0.6
model = training(model,x_train,y_train_std,x_test,y_test_std,lr,512,6)

not none
13.0


/home/markus/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 968303 samples, validate on 110666 samples
Epoch 1/2
0.0005
968303/968303 [==============================] - 157s 162us/step - loss: 0.0206 - val_loss: 0.0188
Epoch 2/2
0.0005
968303/968303 [==============================] - 147s 152us/step - loss: 0.0190 - val_loss: 0.0192
7.0745
7.6097
